In [ ]:
import os
import sys
import glob
import pickle
from scipy.fft import fft, fftfreq
from scipy.signal import butter, filtfilt
from sklearn.metrics import r2_score
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow import keras

if '..' not in sys.path:
    sys.path.append('..')
from dlml.utils import collect_experiments
from dlml.data import load_data_files, load_data_areas

#### Which model to load

In [ ]:
# training on frequency data, 2 output values
# experiment_ID = '9ea493c789b542bf979c51a6031f4044'
# training on frequency data, 4 output values
# experiment_ID = 'f6d9a03f1cfe450288e9cb86da94235f'
# training on time data, 2 output values
experiment_ID = '034a1edb0797475b985f0e1335dab383'
# training on time data, 4 output values
# experiment_ID = 'b346a89d384c4db2ba4058a2c83c4f12'

#### Load the model

In [ ]:
experiments_path = '../experiments/neural_network/'
network_parameters = pickle.load(open(os.path.join(experiments_path, experiment_ID, 'parameters.pkl'), 'rb'))
checkpoint_path = experiments_path + experiment_ID + '/checkpoints/'
checkpoint_files = glob.glob(checkpoint_path + '*.h5')
try:
    epochs = [int(os.path.split(file)[-1].split('.')[1].split('-')[0]) for file in checkpoint_files]
    best_checkpoint = checkpoint_files[epochs.index(np.argmin(val_loss) + 1)]
except:
    best_checkpoint = checkpoint_files[-1]
model = keras.models.load_model(best_checkpoint)
x_train_mean = network_parameters['x_train_mean']
x_train_std  = network_parameters['x_train_std']
x_train_min = network_parameters['x_train_min']
x_train_max = network_parameters['x_train_max']
var_names = network_parameters['var_names']
print(f'Loaded network from {best_checkpoint}.')
print(f'Variable names: {var_names}')

#### Model topology

In [ ]:
model.summary()

Some variables used in the following:

In [ ]:
X, y, Xf = {}, {}, {}
group_index, n_mom_groups = {}, {}

#### Load the original data set

In [ ]:
use_fft = network_parameters['use_fft'] if 'use_fft' in network_parameters else False
if use_fft:
    raise Exception('This notebook must be used on a network that uses time-domain inputs')

set_name = 'test'

data_dir = os.path.join('..', network_parameters['data_dirs'][0].format(network_parameters['area_IDs'][0]))
data_files = sorted(glob.glob(data_dir + os.path.sep + f'*_{set_name}_set.h5'))
ret = load_data_areas({set_name: data_files}, network_parameters['var_names'],
                        network_parameters['generators_areas_map'][:1],
                        network_parameters['generators_Pnom'],
                        network_parameters['area_measure'],
                        trial_dur=network_parameters['trial_duration'],
                        max_block_size=500,
                        use_tf=False, add_omega_ref=True,
                        use_fft=False)
t = ret[0]
X_raw = ret[1][set_name]
y[set_name] = ret[2][set_name]
group_index[set_name] = [np.where(y[set_name] == mom)[0] for mom in np.unique(y[set_name])]

X[set_name] = np.zeros(X_raw.shape)
for i,(m,s) in enumerate(zip(x_train_mean, x_train_std)):
    X[set_name][i,:,:] = (X_raw[i,:,:] - m) / s
X[set_name] = X[set_name].squeeze()
y[set_name] = y[set_name].squeeze()
dt = np.diff(t[:2])[0]
N_samples = t.size
Xf[set_name] = fft(X[set_name])
Xf[set_name] = 2.0 / N_samples * np.abs(Xf[set_name][:, :N_samples//2])
F = fftfreq(N_samples, dt)[:N_samples//2]
n_mom_groups[set_name] = len(group_index[set_name])

#### Load the first data set
Here, the values of inertia of G2 and G3 are changed while keeping the area momentum constant.

In [ ]:
base_folder = network_parameters['data_dirs'][0]
while '{}' in base_folder:
    base_folder,_ = os.path.split(base_folder)
data_files = []
group_index['var_G2_G3'] = []
for i,prefix in enumerate(('low','high')):
    folder = os.path.join('..', base_folder, prefix + '_momentum_' + set_name + '_var_G1_G2')
    files = sorted(glob.glob(folder + os.path.sep + '*.h5'))
    group_index['var_G2_G3'].append(np.arange(len(files)) + len(data_files))
    data_files += files
n_mom_groups['var_G2_G3'] = len(group_index['var_G2_G3'])

ret = load_data_files(data_files,
                      network_parameters['var_names'],
                      network_parameters['generators_areas_map'][:1],
                      network_parameters['generators_Pnom'],
                      'momentum')

X_raw = ret[1][:, :, :-1]
X['var_G2_G3'] = np.zeros(X_raw.shape)
for i,(m,s) in enumerate(zip(x_train_mean, x_train_std)):
    X['var_G2_G3'][i,:,:] = (X_raw[i,:,:] - m) / s
y['var_G2_G3'] = ret[2]
X['var_G2_G3'] = X['var_G2_G3'].squeeze()
y['var_G2_G3'] = y['var_G2_G3'].squeeze()
Xf['var_G2_G3'] = fft(X['var_G2_G3'])
Xf['var_G2_G3'] = 2.0 / N_samples * np.abs(Xf['var_G2_G3'][:, :N_samples//2])

#### Load the second data set
Here, the values of inertia of G4 or G8 are changed. The momentum of area 1 is unchanged.

In [ ]:
base_folder = network_parameters['data_dirs'][0]
while '{}' in base_folder:
    base_folder,_ = os.path.split(base_folder)
generators = 'G4', 'G8'
for gen in generators:
    key = 'var_' + gen
    data_files = []
    group_index[key] = []
    for i,prefix in enumerate(('low','high')):
        folder = os.path.join('..', base_folder, prefix + '_momentum_' + set_name + '_var_' + gen)
        files = sorted(glob.glob(folder + os.path.sep + '*.h5'))
        group_index[key].append(np.arange(len(files)) + len(data_files))
        data_files += files
    n_mom_groups[key] = len(group_index[key])

    ret = load_data_files(data_files,
                          network_parameters['var_names'],
                          network_parameters['generators_areas_map'][:1],
                          network_parameters['generators_Pnom'],
                          'momentum')

    X_raw = ret[1][:, :, :-1]
    X[key] = np.zeros(X_raw.shape)
    for i,(m,s) in enumerate(zip(x_train_mean, x_train_std)):
        X[key][i,:,:] = (X_raw[i,:,:] - m) / s
    y[key] = ret[2]
    X[key] = X[key].squeeze()
    y[key] = y[key].squeeze()
    Xf[key] = fft(X[key])
    Xf[key] = 2.0 / N_samples * np.abs(Xf[key][:, :N_samples//2])

#### Predict the values of area momentum for all the input data sets

In [ ]:
y_pred = {k: model.predict(v) for k,v in X.items()}
ym = {k: [v[group_index[k][i]].mean() for i in range(n_mom_groups[k])] for k,v in y.items()}
ys = {k: [v[group_index[k][i]].std() for i in range(n_mom_groups[k])] for k,v in y.items()}
ym_pred = {k: [v[group_index[k][i]].mean() for i in range(n_mom_groups[k])] for k,v in y_pred.items()}
ys_pred = {k: [v[group_index[k][i]].std() for i in range(n_mom_groups[k])] for k,v in y_pred.items()}

#### Plot the spectra of all the input data sets

In [ ]:
fig,ax = plt.subplots(2, 1, figsize=(6.5, 7))

cmap = plt.get_cmap('Paired')
ax[0].plot(ym['test'], ym['test'], 'ko--', lw=2, markerfacecolor='w')
for i,k in enumerate(ym_pred):
    for j in range(2):
        ax[0].plot(ym[set_name][j] + np.zeros(2),
                ym_pred[k][j] + ys_pred[k][j] * np.array([-1,1]),
                color=cmap(i*2+1), lw=2)
    ax[0].plot(ym[set_name], ym_pred[k], 'o-', color=cmap(i*2+1),
            markerfacecolor='w', markeredgewidth=2, label=k)
ax[0].set_xlabel(r'Exact momentum [GW$\cdot$s$^2$]')
ax[0].set_ylabel(r'Estimated momentum [GW$\cdot$s$^2$]')
ax[0].legend(loc='lower right')
ax[0].grid(which='major', axis='both', lw=0.5, ls=':', color=[.6,.6,.6])
xlim = [0.149, 0.31]
ylim = [0.145, 0.32]
ticks = np.r_[0.15 : 0.31 : 0.03]
ax[0].set_xlim(xlim)
ax[0].set_ylim(ylim)
ax[0].set_xticks(ticks)
ax[0].set_yticks(ticks)

for i,(k,v) in enumerate(Xf.items()):
    for j in range(n_mom_groups[k]):
        m = v[group_index[k][j], :].mean(axis=0)
        s = v[group_index[k][j], :].std(axis=0)
        ci = 1.96 * s / np.sqrt(group_index[k][j].size)
        ax[1].fill_between(F, 20*np.log10(m + ci), 20*np.log10(m - ci),
                        color=cmap(i*2+j), label=k, alpha=0.5)
ax[1].legend(loc='lower left', frameon=False, fontsize=8)
ax[1].set_xscale('log')
ax[1].set_xlabel('Frequency [Hz]')
ax[1].set_ylabel('Power [dB]')

for a in ax:
    for side in 'right','top':
        a.spines[side].set_visible(False)

fig.tight_layout()
fig.savefig(f'variable_inertia_{experiment_ID[:6]}.pdf')